In [1]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model='distilbert-base-uncased-finetuned-sst-2-english')

reviews = [
    "This movie was absolutely fantastic! The acting, the plot, everything was perfect.",
    "I did not enjoy this movie at all. The story was predictable and the acting was terrible.",
    "A solid performance by the lead actor, but the plot could have been better.",
    "What a waste of time! I can't believe I sat through the whole movie."
]

results = sentiment_analysis(reviews)

for review, result in zip(reviews, results):
    print(f"Review: {review}\nSentiment: {result['label']} with score {result['score']:.2f}\n")

/Users/kimdaehyeon/Desktop/tensorflow_test/venv_tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


Review: This movie was absolutely fantastic! The acting, the plot, everything was perfect.
Sentiment: POSITIVE with score 1.00

Review: I did not enjoy this movie at all. The story was predictable and the acting was terrible.
Sentiment: NEGATIVE with score 1.00

Review: A solid performance by the lead actor, but the plot could have been better.
Sentiment: POSITIVE with score 0.68

Review: What a waste of time! I can't believe I sat through the whole movie.
Sentiment: NEGATIVE with score 1.00



In [2]:
from datasets import load_dataset
dataset = load_dataset("imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"],truncation = True,padding ="max_length", max_length=512)

tokenized_datasets = dataset.map(preprocess_function,batched=True)

In [4]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

In [7]:
trainer.train()

KeyboardInterrupt: 

In [8]:
results = trainer.evaluate()
prunt(results)

RuntimeError: MPS backend out of memory (MPS allocated: 8.55 GB, other allocations: 478.88 MB, max allowed: 9.07 GB). Tried to allocate 96.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
model_path = ".fine_tuned_distilbert_imdb"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

model.eval()

In [ ]:
import torch

input_text = "This movie was absolutely fantastic!!"
inputs = tokenizer(input_text, return_tensors="pt")

with torch.no_glad():
    outputs = model(**inputs)

logits = outputs.logits
predicted_class_id = torch.argmax(logits,dim=-1).item()

print(f"Predicted class ID : {predicted_class_id}" )